# 1. Bilder für Identification vorbereiten

In diesem Notebook werden Schritt für Schritt die notwendigen Operationen durchgeführt, um die Bilder für das Trainieren eines Klassifikators vorzubereiten. Eingangs muss der Pfad für die eingehenden Bilder richtig gesetzt werden. 

In [1]:
import os
import torch
import numpy as np

from PIL import Image, ImageOps
from torchvision import transforms as tr

from matplotlib import pyplot as plt
import matplotlib.patches as patches

# Konstanten


## 1.1. Bilder padden und resizen
Damit die Bilder für das bestimmen der Bounding-Boxes verwendet werden können, müssen sie zunächt auf die richtige Größe gebracht werden. Um die Seitenverhältnisse nicht zu verzerren, werden die Originalbilder links und rechts mit einem Padding versehen und dann auf die richtig Größe (300x300 Pixel) angepasst. 

Weiterhin ist wichtig, dass die Ordnerstruktur berücksichtigt wird. Die Unterordner im data-Verzeichnis enthalten immer jeweils Bilder eine individuellen Kuh.

TODO: Pfad zum full_sorted Ordner erklaeren und verlinken

In [3]:
IMG_WIDTH	= 300
IMG_HEIGHT	= 300

INPUT_IMGS_PATH  = 'C:/dev/netcase/cow_dataset/full_sorted/'
OUTPUT_IMGS_PATH = 'data/pad_resize/'

# Output Ordner erstellen, falls er nicht existiert
if not os.path.isdir(OUTPUT_IMGS_PATH):
    os.mkdir(OUTPUT_IMGS_PATH)

for img_dir in os.listdir(INPUT_IMGS_PATH):
    if os.path.isdir(os.path.join(INPUT_IMGS_PATH, img_dir)):
        for img_file in os.listdir(os.path.join(INPUT_IMGS_PATH, img_dir)):
            
            input_path = os.path.join(INPUT_IMGS_PATH, img_dir, img_file)
            
            # Ordner und Nicht-Bild Dateien überspringen
            if not os.path.isdir(input_path) and os.path.splitext(input_path)[-1].lower() == ".jpg":
            
                # Vorbereiten der Bilder
                image = Image.open(input_path)
                image = ImageOps.pad(image, (IMG_WIDTH, IMG_HEIGHT))
                image = image.rotate(90)
                
                # Ausgabeordner für jeweilige Kuh anlegen
                output_dir = os.path.join(OUTPUT_IMGS_PATH, img_dir)
                if not os.path.isdir(output_dir):
                    os.mkdir(output_dir)
                
                # Bild speichern
                output_path = os.path.join(OUTPUT_IMGS_PATH, img_dir, img_file)
                image.save(output_path)

                image.close()

## 1.2. Bounding-Boxes bestimmen und extrahieren
Hou et al. (TODO: put link) nutzen in ihrer Implementation des Systems einen Single Shot Multibox Detector (SSD). Diesen selbst aufzusetzen und zu trainieren ist für diese Beipiel allerdings nicht zielführend. NVIDIA bietet hier einen Out-Of-The-Box Lösung (TODO: put link), die wir anwenden, um die Bounding-Boxes zu bestimmen. Da der Detector am COCO Dataset trainiert wurde, kann er Kühe bereits erkennen und benötigt dementsprechend keine weiteren Anpassungen mehr.

Der folgende Code ist stark an die Pytorch Anleitung (TODO: put link) angelehnt und unterscheidet sich hauptsächlich darin, dass die generierten Bounding-Boxes genutzt werden, um die Bildausschnitte zu extrahieren.

In [12]:
PAD_IMGS_PATH = 'data/pad_resize/'
BOX_IMGS_PATH = 'data/bounding_box/'

# create output directory if it does not exist
if not os.path.isdir(BOX_IMGS_PATH):
    os.mkdir(BOX_IMGS_PATH)

# Laden des Models und zusätzlicher Utility 
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

# Konfiguration des Models 
ssd_model.to('cuda')
ssd_model.eval()

show_one = True

# Pro Kuh, alle Bounding Boxes in den Bildern bestimmen 
for img_dir in os.listdir(PAD_IMGS_PATH):
    # Liste aller Bild-Pfade
    uris = []

    # Alles Bild-Pfade im Ordner laden 
    for img_file in os.listdir(os.path.join(PAD_IMGS_PATH, img_dir)):
        uris.append(os.path.join(PAD_IMGS_PATH, img_dir, img_file))
    
    # Vorbereiten der Bilder (vgl. Tutorial) (TODO: put link)
    inputs = [utils.prepare_input(uri) for uri in uris]
    tensor = utils.prepare_tensor(inputs)

    with torch.no_grad():
        detections_batch = ssd_model(tensor)

    # Resultate verarbeiten (vgl. Tutorial) (TODO: put link)
    results_per_input = utils.decode_results(detections_batch)
    best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]
    classes_to_labels = utils.get_coco_object_dictionary()

    # TODO: show a bounding box here as example

    for image_idx in range(len(best_results_per_input)):
        # Originales, wiederhergestelltes Bild
        image = inputs[image_idx] / 2 + 0.5
        bboxes, classes, confidences = best_results_per_input[image_idx]

        # Bestes Match
        max_index = confidences.argmax()
        
        # Bounding Box extrahieren
        # NOTE: Im Vergleich zum Tutorial mussten top und bot getauscht werden, da y=0 normalerweise unten ist.
        left, top, right, bot = bboxes[max_index] * 300

        # NOTE: Die Pixelwerte in image sind floats und müssen erst in 8-Bit ints umgewandelt werden.
        output_image = Image.fromarray((image * 255).astype(np.uint8))
        output_image = output_image.crop((left, top, right, bot))

        # Zurückschreiben
        head, tail = os.path.split(img_dir)
        
        output_dir = os.path.join(BOX_IMGS_PATH, tail + '/')
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)

        output_path = os.path.join(BOX_IMGS_PATH, tail + '/', str(image_idx) + '.jpg')
        output_image.save(output_path)
        output_image.close()

    # NOTE: Die Implementation vom SSD scheint den GPU Speicher nicht eigentständig freizugeben. Dementsprechend wird er hier explizit geleert.
    torch.cuda.empty_cache()

Using cache found in C:\Users\Christian/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\Christian/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


[0.33465755 0.0676491  0.7200849  0.95493376]


## 1.3. Image Dataset Augmentation
Um ausreichendes Bildmaterial zur Verfügung zu haben und Probleme wie Overfitting zu vermeiden, werden nun auf den Datensatz einfache geometrische Transformationen angewendet. Dies ist eine gängige Methode beim Trainieren von Neuronalen Netzen und wurde auch in Hou et al. (TODO: put link) durchgeführt. Hier wurde der Datensatz um das 20-fache vergrößert (TODO: put absolut number as well).

In diesem Beispiel wird der Datensatz auf das 8-fache vergrößert.

In [3]:
# TODO: put all constants up to the top and don't re-define them
BOX_IMGS_PATH = 'data/bounding_box/'
AUG_IMG_PATH  = 'data/augment/'

# Anzahl der Kopien pro Bild
AUGMENT_FACTOR = 8

# Erstellen des Verzeichnises, falls es nicht existiert
if not os.path.isdir(AUG_IMG_PATH):
    os.mkdir(AUG_IMG_PATH)

# Pipeline, um Bilder zu flippen und/oder verzerren 
applier = tr.RandomApply(transforms=[tr.RandomHorizontalFlip(), 
                                     tr.RandomPerspective(distortion_scale=0.6, p=1), 
                                     tr.RandomResizedCrop(size=(224, 224), scale=(0.7, 1.0))], p=1)

for img_dir in os.listdir(BOX_IMGS_PATH):
    img_index = 0
    for img_file in os.listdir(os.path.join(BOX_IMGS_PATH, img_dir)):
        to_augment = Image.open(os.path.join(BOX_IMGS_PATH, img_dir, img_file))

        # Originales Bild speichern
        if not os.path.isdir(os.path.join(AUG_IMG_PATH, img_dir)):
            os.mkdir(os.path.join(AUG_IMG_PATH, img_dir))

        to_augment.save(os.path.join(AUG_IMG_PATH, img_dir, img_file))

        # Augmentation anwenden
        augmented_imgs = [applier(to_augment) for _ in range(AUGMENT_FACTOR)] 
        
        # Augmented Bilder speichern
        for aug_index in range(len(augmented_imgs)):
            img_path = os.path.join(AUG_IMG_PATH, img_dir, 'aug_' + str(img_index) + '_' + str(aug_index) + '.jpg')
            augmented_imgs[aug_index].save(img_path)

        img_index += 1
        to_augment.close()